<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C1/W4/ungraded_labs/C1_W4_Lab_3_compacted_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download the training set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

In [ ]:
# Download the validation set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

Then unzip them:

In [ ]:
import zipfile
local_zip1 = '/content/horse-or-human.zip'
local_zip2 = '/content/validation-horse-or-human.zip'
zip_ref1 = zipfile.ZipFile(local_zip1, 'r')
zip_ref1.extractall('/content/horse-or-human')
zip_ref1.close()
zip_ref2 = zipfile.ZipFile(local_zip2, 'r')
zip_ref2.extractall('/content/validation-horse-or-human')
zip_ref2.close()


Then define the directories containing the images:

In [ ]:
import os
train_horse_dir = os.path.join('/content/horse-or-human/horses')
train_human_dir = os.path.join('/content/horse-or-human/humans')
test_horse_dir = os.path.join('/content/validation-horse-or-human/horses')
test_human_dir= os.path.join('/content/validation-horse-or-human/humans')


You can check that the directories are not empty and that the train set has more images than the validation set:

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
train_human_names = os.listdir(train_human_dir)
test_human_names = os.listdir(test_human_dir)
test_horse_names = os.listdir(test_horse_dir)

In [ ]:
print(len(train_horse_names))
print(len(train_human_names))
print(len(test_horse_names))
print(len(test_human_names))

## Build the Model

The model will follow the same architecture as before but they key difference is in the `input_shape` parameter of the first `Conv2D` layer. Since you will be compacting the images later in the generator, you need to specify the expected image size here. So instead of 300x300 as in the previous two labs, you specify a smaller 150x150 array.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
fname = train_horse_names[0]
os_sample_path = os.path.join(train_horse_dir, fname)
sample = mpimg.imread(os_sample_path)
plt.imshow(sample)
plt.show()

In [ ]:
sample.shape

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3,3), input_shape = (300, 300,3), activation= 'relu'), tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Conv2D(32, (3,3), activation= 'relu'), tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Conv2D(32, (3,3), activation= 'relu'), tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Conv2D(64, (3,3),  activation= 'relu'), tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Conv2D(64, (3,3),  activation= 'relu'), tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(256, activation = 'relu'),
                                    tf.keras.layers.Dense(1, activation = 'sigmoid')
                                    ])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1/255)
train_datagenerator = train_datagen.flow_from_directory(
    '/content/horse-or-human',
    batch_size = 128,
    target_size = (300, 300),
    class_mode = 'binary'
)

test_datagen = ImageDataGenerator(rescale = 1/255)
test_datagenerator = test_datagen.flow_from_directory(
    '/content/validation-horse-or-human',
    batch_size = 32,
    target_size = (300, 300),
    class_mode = 'binary'
)





In [ ]:
model.compile(tf.optimizers.RMSprop(learning_rate = 0.001), metrics = 'accuracy', loss = 'mean_squared_error')

In [ ]:
history = model.fit(
    train_datagenerator,
    steps_per_epoch = 8,
    epochs = 15,
    validation_data = test_datagenerator,
    validation_steps = 8,
    verbose = 1
)

In [ ]:
import numpy as np

In [ ]:
from google.colab import files
from tensorflow.keras.utils import load_img,img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = load_img(path, target_size = (300, 300))
  x = img_to_array(img)
  x = x/255
  x = np.expand_dims(x, axis = 0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size = 10)
  if classes[0]>0.5:
    print(f'{fn} is human')
  else :
    print(f'{fn} is horse')


In [ ]:
from google.colab import runtime
runtime.unassign()